In [1]:
import glob
import os
import pickle as pkl
import re
import sys

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [15]:
with open('../../datasets/khazana/khazana_df.pkl', "rb") as f:
    khaz_db = pkl.load(f)

In [16]:
cols = pd.MultiIndex.from_arrays([khaz_db.columns, khaz_db.iloc[0]], names=["label","units"])
khaz_db.columns = cols
khaz_db

label,ID,Title,Natoms,Ntypes,Volume of Cell,"Band Gap, PBE","Band Gap, HSE06","Band Gap, Experimental",Dielectric Constant,"Dielectric Constant, Electronic",...,Density,Magnetization,Spontaneous Polarization,Refractive Index,Decomposition Energy,Formation Energy,Ionization Energy,Electron Affinity,Shear Modulus,Cohesive Energy
units,NaN,NaN,NaN,NaN,(Å3),(eV),(eV),(eV),NaN,NaN,...,(g/cm3),(μB),(μC/cm2),NaN,(eV),(eV),(eV),(eV),(GPa),(eV/atom)
0,NaN,NaN,NaN,NaN,(Å3),(eV),(eV),(eV),NaN,NaN,...,(g/cm3),(μB),(μC/cm2),NaN,(eV),(eV),(eV),(eV),(GPa),(eV/atom)
1,1245.0,CH6I3NSn,12.0,5.0,NaN,0.2,1.22,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1246.0,CH6I3NSn,12.0,5.0,NaN,1.3,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1247.0,SnI3NH4,9.0,4.0,NaN,0.4,1.65,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1248.0,SnI3NH4,9.0,4.0,NaN,1.9,3.05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,NaN,Imidazolium Lead Iodide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2695,3195.0,C3H5N2PbI3,14.0,5.0,280.69,2.23,3.1,NaN,12.01,4.52,...,3.89,NaN,NaN,2.13,NaN,0.41,NaN,NaN,NaN,NaN
2696,NaN,Imidazolium Lead Iodide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
valid_idxs = khaz_db.drop(index=0)["ID"].dropna().index
khaz_useful = khaz_db.loc[valid_idxs]
khaz_useful

label,ID,Title,Natoms,Ntypes,Volume of Cell,"Band Gap, PBE","Band Gap, HSE06","Band Gap, Experimental",Dielectric Constant,"Dielectric Constant, Electronic",...,Density,Magnetization,Spontaneous Polarization,Refractive Index,Decomposition Energy,Formation Energy,Ionization Energy,Electron Affinity,Shear Modulus,Cohesive Energy
units,NaN,NaN,NaN,NaN,(Å3),(eV),(eV),(eV),NaN,NaN,...,(g/cm3),(μB),(μC/cm2),NaN,(eV),(eV),(eV),(eV),(GPa),(eV/atom)
1,1245.0,CH6I3NSn,12.0,5.0,NaN,0.2,1.22,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1246.0,CH6I3NSn,12.0,5.0,NaN,1.3,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1247.0,SnI3NH4,9.0,4.0,NaN,0.4,1.65,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1248.0,SnI3NH4,9.0,4.0,NaN,1.9,3.05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1249.0,CH5I3N2Sn,12.0,5.0,NaN,0.2,1.2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689,3192.0,C3H5N2PbI3,14.0,5.0,285.78,1.53,2.35,NaN,43.32,5.20,...,3.82,NaN,NaN,2.28,NaN,0.41,NaN,NaN,NaN,NaN
2691,3193.0,C3H5N2PbI3,14.0,5.0,285.83,1.55,2.39,NaN,72.69,5.25,...,3.82,NaN,NaN,2.29,NaN,0.41,NaN,NaN,NaN,NaN
2693,3194.0,C3H5N2PbI3,14.0,5.0,285.67,1.69,2.51,NaN,28.55,5.17,...,3.82,NaN,NaN,2.27,NaN,0.41,NaN,NaN,NaN,NaN


In [27]:
with open("khaz_useful.pkl", "wb") as wf:
    pkl.dump(khaz_useful, wf)

In [31]:
khaz_useful["Dielectric Constant"].dropna(axis=0)

units,NaN
7,5.40
9,5.61
11,6.11
13,5.30
15,4.96
...,...
2689,43.32
2691,72.69
2693,28.55
2695,12.01
